In [1]:
!pip install -U ydata-profiling
!pip install --upgrade jupyter ipywidgets
!pip install openpyxl

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.3/357.3 kB 3.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 12.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 15.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 14.2 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.4/455.4 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [2]:
import pandas as pd

df1 = pd.read_excel('/home/aistudio/data/data237885/附件1.xlsx')
df2 = pd.read_excel('/home/aistudio/data/data237885/附件2.xlsx')
df3 = pd.read_excel('/home/aistudio/data/data237885/附件3.xlsx')
df4 = pd.read_excel('/home/aistudio/data/data237885/附件4.xlsx')

In [3]:
import calendar
import numpy as np

# 使用单品编码进行合并，并且只保留附件2中已有的单品编码
merged_df = pd.merge(df2, df1[['单品编码', '单品名称', '分类名称']], on='单品编码', how='left')

merged_df = pd.merge(merged_df, df3[['日期', '单品编码', '批发价格(元/千克)']], left_on=['销售日期', '单品编码'], right_on=['日期', '单品编码'], how='left')

# 添加一个序号列，从1开始
merged_df['序号'] = np.arange(1, len(merged_df) + 1)

# 如果你想让'序号'列成为第一列，你可以这样重新排列列的顺序：
cols = ['序号'] + [col for col in merged_df.columns if col != '序号']
merged_df = merged_df[cols]

# 转为日期格式
merged_df['销售日期'] = pd.to_datetime(merged_df['销售日期'])

# 添加“星期”列（1-7的数字表示）
merged_df['星期'] = pd.to_datetime(merged_df['销售日期']).dt.dayofweek + 1  # +1是为了让星期一开始为1

# 添加“年中进度”列
merged_df['年中进度'] = merged_df['销售日期'].apply(
    lambda x: x.timetuple().tm_yday / (366 if calendar.isleap(x.year) else 365)
)

# 新增一个表示所在周的列
merged_df['周'] = merged_df['销售日期'].dt.to_period('W')

# 新增一个表示所在月份的列
merged_df['月份'] = merged_df['销售日期'].dt.to_period('M')

# 创建一个新的列"季度"
merged_df['季度'] = merged_df['销售日期'].dt.to_period('Q')

In [4]:
def time_to_day_fraction(time_str):
    from datetime import datetime
    try:
        time_obj = datetime.strptime(time_str, '%H:%M:%S.%f').time()
    except ValueError:
        time_obj = datetime.strptime(time_str, '%H:%M:%S').time()

    total_seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second + time_obj.microsecond * 1e-6
    return total_seconds / 86400  # 86400 = 24*60*60

merged_df['扫码销售时间'] = merged_df['扫码销售时间'].apply(time_to_day_fraction)

In [5]:
# 将销售日期转换为字符串格式，以保持原来的格式（只包含日期部分）
merged_df['销售日期'] = merged_df['销售日期'].dt.strftime('%Y-%m-%d')

# 添加“交易金额”列
merged_df['交易金额'] = merged_df['销量(千克)'] * merged_df['销售单价(元/千克)']

# 计算“利润”列
merged_df['利润'] = merged_df['交易金额'] - (merged_df['销量(千克)'] * merged_df['批发价格(元/千克)'])

# 将“销售类型”列中的“销售”赋值为1，“退货”赋值为-1
merged_df['销售类型'] = merged_df['销售类型'].replace({'销售': 1, '退货': -1})

# 将“是否打折销售”列中的“是”赋值为1，“否”赋值为0
merged_df['是否打折销售'] = merged_df['是否打折销售'].replace({'是': 1, '否': 0})

In [6]:
merged_df.drop('日期', axis=1, inplace=True)

In [7]:
def QutlierReplacement(df, col, percentage=0.02):
    """
    传入DataFrame对象和需要处理异常值的列名。
    如果异常值的数据量小于数据量*百分比，就替换数据。
    """
    processed_records = {}  # 用于存储被处理的记录
    
    df = df.reset_index(drop=True)
    df_col = df[col]  # 获取列
    df_col_value = df[col].values
    
    # 计算下四分位数和上四分位数
    Q1 = df_col.quantile(q=0.25)
    Q3 = df_col.quantile(q=0.75)
    
    # 基于5倍的四分位差计算上对应的值
    up_whisker = Q3 + 5 * (Q3 - Q1)
    
    right = set(np.where(df_col_value > up_whisker)[0])  # 只考虑极大的异常值
    
    choose = list(right)
    
    if len(choose) < len(df) * percentage:
        for idx in choose:
            original_value = df.at[idx, col]
            item_name = df.at[idx, '单品名称']
            
            # 计算该单品名称下的销量(千克)的平均值
            mean_value = df[df['单品名称'] == item_name][col].mean()
            
            df.at[idx, col] = mean_value  # 使用该单品名称下的平均值进行替换
            processed_records[df.at[idx, '序号']] = (original_value, df.at[idx, col])  # 存储被处理的记录
    
    return df, processed_records  # 返回处理异常值后的数据和被处理的记录

# 使用该函数处理merged_df中的“销量(千克)”列
merged_df, processed_records = QutlierReplacement(merged_df, '销量(千克)')

count = 0
# 输出被处理的记录
for seq_no, (original_value, new_value) in processed_records.items():
    print(f"序号: {seq_no}, 处理前销量(千克): {original_value}, 处理后销量(千克): {new_value}")
    count += 1
    
print(count)

序号: 606217, 处理前销量(千克): 5.0, 处理后销量(千克): 4.2408906882591095
序号: 638988, 处理前销量(千克): 5.0, 处理后销量(千克): 4.2393540297333185
序号: 628749, 处理前销量(千克): 5.0, 处理后销量(千克): 4.237814260562738
序号: 618515, 处理前销量(千克): 5.0, 处理后销量(千克): 4.236271374450517
序号: 630803, 处理前销量(千克): 5.0, 处理后销量(千克): 4.234725365087057
序号: 606231, 处理前销量(千克): 5.0, 处理后销量(千克): 4.233176226149985
序号: 618519, 处理前销量(千克): 5.0, 处理后销量(千克): 4.231623951304134
序号: 612380, 处理前销量(千克): 5.0, 处理后销量(千克): 4.2300685342015125
序号: 616476, 处理前销量(千克): 5.0, 处理后销量(千克): 4.228509968481272
序号: 614430, 处理前销量(千克): 10.0, 处理后销量(千克): 4.226948247769696
序号: 581667, 处理前销量(千克): 4.141, 处理后销量(千克): 0.447347337204992
序号: 620583, 处理前销量(千克): 5.0, 处理后销量(千克): 4.215261908190282
序号: 157740, 处理前销量(千克): 7.551, 处理后销量(千克): 1.2641466596389512
序号: 649260, 处理前销量(千克): 5.0, 处理后销量(千克): 4.213673369542894
序号: 157742, 处理前销量(千克): 9.694, 处理后销量(千克): 1.2637324513545682
序号: 694316, 处理前销量(千克): 3.03, 处理后销量(千克): 1.0119750312109863
序号: 655414, 处理前销量(千克): 5.0, 处理后销量(千克): 4.212081615230227
序号: 254012, 处理前销量

In [8]:
merged_df.head()

,序号,销售日期,扫码销售时间,单品编码,销量(千克),销售单价(元/千克),销售类型,是否打折销售,单品名称,分类名称,批发价格(元/千克),星期,年中进度,周,月份,季度,交易金额,利润
0,1,2020-07-01,0.385508,102900005117056,0.396,7.6,1,0,泡泡椒(精品),辣椒类,4.32,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,3.0096,1.29888
1,2,2020-07-01,0.387121,102900005115960,0.849,3.2,1,0,大白菜,花叶类,2.10,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,2.7168,0.93390
2,3,2020-07-01,0.387198,102900005117056,0.409,7.6,1,0,泡泡椒(精品),辣椒类,4.32,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,3.1084,1.34152
3,4,2020-07-01,0.388720,102900005115823,0.421,10.0,1,0,上海青,花叶类,7.03,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,4.2100,1.25037
4,5,2020-07-01,0.389163,102900005115908,0.539,8.0,1,0,菜心,花叶类,4.60,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,4.3120,1.83260


In [9]:
merged_df.describe()

,序号,扫码销售时间,单品编码,销量(千克),销售单价(元/千克),销售类型,是否打折销售,批发价格(元/千克),星期,年中进度,交易金额,利润
count,878503.000000,878503.000000,8.785030e+05,878503.000000,878503.000000,878503.000000,878503.000000,878503.000000,878503.000000,878503.000000,878503.000000,878503.000000
mean,439252.000000,0.621333,1.030313e+14,0.533810,8.917144,0.998950,0.053917,5.663150,4.268100,0.505362,3.835805,1.415649
std,253602.116104,0.152056,7.174368e+11,0.336168,6.311265,0.045803,0.225853,4.229638,2.041563,0.287668,2.813984,1.249791
min,1.000000,0.343202,1.029000e+14,-9.082000,0.100000,-1.000000,0.000000,0.010000,1.000000,0.002740,-100.000000,-161.900000
25%,219626.500000,0.467446,1.029000e+14,0.290000,4.900000,1.000000,0.000000,3.000000,2.000000,0.246575,2.210000,0.733405
50%,439252.000000,0.642595,1.029000e+14,0.435000,7.900000,1.000000,0.000000,4.690000,4.000000,0.538251,3.332000,1.195600
75%,658877.500000,0.752034,1.029000e+14,0.741000,10.800000,1.000000,0.000000,7.010000,6.000000,0.745902,4.791600,1.859550
max,878503.000000,0.966528,1.069740e+14,4.878049,119.900000,1.000000,1.000000,141.000000,7.000000,1.000000,944.000000,420.800000


In [10]:
# 将合并后的DataFrame保存为新的Excel文件
merged_df.to_excel("/home/aistudio/data/data237885/附件2(处理后).xlsx", index=False)

In [11]:
# 检查“销售类型”是“销售”，但“销量(千克)”不是正数的情况
mask1 = (merged_df['销售类型'] == '销售') & (merged_df['销量(千克)'] <= 0)
# 检查“销售类型”是“退货”，但“销量(千克)”不是负数的情况
mask2 = (merged_df['销售类型'] == '退货') & (merged_df['销量(千克)'] >= 0)

# 将不符合条件的行选出来
invalid_rows1 = merged_df[mask1]
invalid_rows2 = merged_df[mask2]

if len(invalid_rows1) > 0:
    print("以下行的‘销售类型’是‘销售’，但‘销量(千克)’不是正数：")
    print(invalid_rows1)
if len(invalid_rows2) > 0:
    print("以下行的‘销售类型’是‘退货’，但‘销量(千克)’不是负数：")
    print(invalid_rows2)

if len(invalid_rows1) == 0 and len(invalid_rows2) == 0:
    print("所有数据都满足条件。")

所有数据都满足条件。


In [41]:
# 转换'销售日期'列为日期格式，并提取月份信息
merged_df['销售日期'] = pd.to_datetime(merged_df['销售日期'])
merged_df['月份'] = merged_df['销售日期'].dt.to_period('M')  # 将日期转换为月份

# 按照月份和分类名称分组，然后计算每组的“销量(千克)”和“利润”的总和
grouped_df = merged_df.groupby(['月份', '分类名称']).agg({'销量(千克)': 'sum', '利润': 'sum'}).reset_index()

# 转换为宽格式
pivot_df = grouped_df.pivot(index='月份', columns='分类名称', values=['销量(千克)', '利润'])

# 重命名列名以便于理解
pivot_df.columns = [f"{col[1]}{col[0]}" for col in pivot_df.columns]

# 将月份列的数据类型转换为字符串，并格式化为"YYYY-MM"格式
pivot_df.index = pivot_df.index.strftime('%Y-%m')

# 保存为新的Excel文件
pivot_df.to_excel("/home/aistudio/data/data237885/每月各品类销量及利润分析.xlsx")

In [49]:
# 按照销售日期、星期和分类名称分组，然后计算每组的“销量(千克)”和“利润”的总和
grouped_df = merged_df.groupby(['销售日期', '星期', '分类名称']).agg({'销量(千克)': 'sum', '利润': 'sum'}).reset_index()

# 转换为宽格式
pivot_df = grouped_df.pivot(index=['销售日期', '星期'], columns='分类名称', values=['销量(千克)', '利润'])

# 重命名列名以便于理解
pivot_df.columns = [f"{col[1]}_{col[0]}" for col in pivot_df.columns]

# 由于索引现在是多层次的（销售日期和星期），我们只将销售日期列的数据类型转换为字符串，并格式化为"YYYY-MM-DD"格式
pivot_df.index = pd.MultiIndex.from_tuples([(idx[0].strftime('%Y-%m-%d'), idx[1]) for idx in pivot_df.index])

# 保存为新的Excel文件
pivot_df.to_excel("/home/aistudio/data/data237885/每日各品类销量及利润分析.xlsx")

In [43]:
# 创建一个日期范围从2020-07-01到2023-06-30
all_dates = pd.date_range(start='2020-07-01', end='2023-06-30')

# 查找在'销售日期'列中存在的日期
existing_dates = pd.to_datetime(merged_df['销售日期'].unique())

# 使用Pandas的Index差集功能来找到缺失的日期
missing_dates = all_dates.difference(existing_dates)

# 打印或输出缺失的日期
print("缺失的日期如下：")
print(missing_dates)

缺失的日期如下：
DatetimeIndex(['2021-02-11', '2021-02-12', '2022-01-31', '2022-11-02',
               '2022-11-04', '2022-11-30', '2022-12-01', '2022-12-02',
               '2022-12-03', '2023-01-21'],
              dtype='datetime64[ns]', freq=None)


## 一键快速预览数据

import numpy as np
from ydata_profiling import ProfileReport

#创建数据分析报告
profile = ProfileReport(merged_df, title='Your Data Analysis Report', explorative=True)

#保存报告为HTML文件
profile.to_file("/home/aistudio/data/data237885/merged_ydata_report.html")

#显示报告（适用于Jupyter Notebook）
profile.to_notebook_iframe()

In [12]:
from datetime import datetime

# 定义数据中最早和最晚的两个日期字符串
date_str1 = '2020-07-01 00:00:00'
date_str2 = '2023-06-30 00:00:00'

# 将日期字符串转换为 datetime 对象
date1 = datetime.strptime(date_str1, '%Y-%m-%d %H:%M:%S')
date2 = datetime.strptime(date_str2, '%Y-%m-%d %H:%M:%S')

# 计算两个日期之间的天数差
delta = date2 - date1

# 输出天数
print(f"间隔天数: {delta.days}（天）")

间隔天数: 1094（天）


In [34]:
temp_merged_df = merged_df.copy()

print(temp_merged_df.shape)

(878503, 18)


In [24]:
# Step 1: 创建 temp_merged_df 作为 merged_df 的副本
temp_merged_df = merged_df.copy()

# Step 2: 按照“销售日期”和“单品名称”进行分组，计算每天各单品的总“销量(千克)”
grouped_df = temp_merged_df.groupby(['销售日期', '单品名称'])['销量(千克)'].sum().reset_index()

# Step 3: 使用 df1 中的全部单品名称，确保所有单品都在结果中出现
# 创建一个唯一的销售日期列表和单品名称列表
unique_dates = temp_merged_df['销售日期'].unique()
unique_products = df1['单品名称'].unique()

# 创建一个全排列的 DataFrame，其中包含所有可能的“销售日期”和“单品名称”组合
import itertools
all_combinations = pd.DataFrame(list(itertools.product(unique_dates, unique_products)), columns=['销售日期', '单品名称'])

# 将全排列的 DataFrame 与 grouped_df 合并，以获取每个单品每天的销量（如果没有记录，则为 0）
final_df = pd.merge(all_combinations, grouped_df, on=['销售日期', '单品名称'], how='left').fillna(0)

# 添加“分类名称”列
final_df = pd.merge(final_df, df1[['单品名称', '分类名称']], on='单品名称', how='left')

output_path = '/home/aistudio/data/data237885/每日各单品总销量情况.xlsx'
final_df.to_excel(output_path, index=False)